# DAAN 862 Lesson 5 Assignment 1

In [242]:
import pandas as pd

## Load, explore and clean the datases

In [245]:
# Step 1: Load the datasets
registration_df = pd.read_csv('Registration.csv')
course_info_df = pd.read_excel('Course_info.xlsx')

In [247]:
# Display basic information about the registration dataframe
print(registration_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4900 entries, 0 to 4899
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Student name  4900 non-null   object
 1   semester new  4900 non-null   object
 2   coursename    4899 non-null   object
dtypes: object(3)
memory usage: 115.0+ KB
None


In [249]:
# Check for missing values
print(registration_df.isnull().sum())

Student name    0
semester new    0
coursename      1
dtype: int64


In [251]:
# Handle missing values
registration_df.dropna(inplace=True)

In [253]:
registration_df.columns = registration_df.columns.str.lower().str.replace(' ', '_')

In [255]:
# Convert semester to a standard format
registration_df['semester'] = pd.to_datetime(registration_df['semester_new'].str.split().str[1] + 
                                             registration_df['semester_new'].str.split().str[0].map({'Spring': '03', 'Summer': '06', 'Fall': '09'}),
                                             format='%Y%m')

In [257]:
# Remove the original 'semester_new' column
registration_df.drop('semester_new', axis=1, inplace=True)

In [259]:
# Display the first few rows of the cleaned dataframe
print(registration_df.head())

  student_name                  coursename   semester
0    Bill Mumy     BEHAVIORAL PHARMACOLOGY 2004-09-01
1    Bill Mumy     AMERICAN FOREIGN POLICY 2000-09-01
2    Bill Mumy       DRUGS, BRAIN AND MIND 2003-09-01
3    Bill Mumy  Environmental Case Studies 2005-09-01
4    Bill Mumy     COMPUTER LINEAR ALGEBRA 2000-09-01


In [261]:
# Check for missing values
print(registration_df.isnull().sum())

student_name    0
coursename      0
semester        0
dtype: int64


In [263]:
# Display basic information about the course info dataframe
print(course_info_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Course number  42 non-null     object
 1   Course Name    41 non-null     object
 2   Course Type    42 non-null     object
dtypes: object(3)
memory usage: 1.1+ KB
None


In [265]:
# Check for missing values
print(course_info_df.isnull().sum())

Course number    0
Course Name      1
Course Type      0
dtype: int64


In [267]:
# Handle missing values
course_info_df.dropna(inplace=True)

In [269]:
# Standardize column names
course_info_df.columns = course_info_df.columns.str.lower().str.replace(' ', '_')


In [271]:
# Check for missing values
print(course_info_df.isnull().sum())

course_number    0
course_name_     0
course_type      0
dtype: int64


In [273]:
# Display the first few rows of the cleaned dataframe
print(course_info_df.head())

  course_number                                     course_name_ course_type
0       ARTS400  EXPERIMENTAL WRITING SEM: The Ecology of Poetry           C
1       ARTS401                             ART: ancient to 1945           C
2       ARTS465                         ENVIRONMENTAL SYSTEMS II           F
3       ARTS486                          COMPUTER LINEAR ALGEBRA           F
4       ARTS512                             ANALYTICAL MECHANICS           F


## Which course has the highest registration? 

In [276]:
# Count the number of registrations for each course
course_registration_count = registration_df['coursename'].value_counts()

# Get the course with the highest registration
highest_registration_course = course_registration_count.index[0]
highest_registration_count = course_registration_count.iloc[0]

print(f"The course with the highest registration is '{highest_registration_course}' with {highest_registration_count} registrations.")

The course with the highest registration is 'COMPUT LINEAR ALGEBRA' with 411 registrations.


## Inner Join two datasets

In [279]:
# Print column names of the Registration DataFrame
print("Columns in Registration DataFrame:")
print(registration_df.columns)

Columns in Registration DataFrame:
Index(['student_name', 'coursename', 'semester'], dtype='object')


In [281]:
# Print column names of the Course Info DataFrame
print("Columns in Course Info DataFrame:")
print(course_info_df.columns)

Columns in Course Info DataFrame:
Index(['course_number', 'course_name_', 'course_type'], dtype='object')


In [283]:
# Perform inner join
merged_df = pd.merge(registration_df, course_info_df, 
                     left_on='coursename', 
                     right_on='course_name_', 
                     how='inner')

In [285]:
# Display the first few rows of the merged dataframe
print(merged_df.head())

  student_name               coursename   semester course_number  \
0    Bill Mumy  BEHAVIORAL PHARMACOLOGY 2004-09-01       ARTS516   
1    Bill Mumy  COMPUTER LINEAR ALGEBRA 2000-09-01       ARTS486   
2    Bill Mumy     ANALYTICAL MECHANICS 2002-09-01       ARTS512   
3    Bill Mumy     ANALYTICAL MECHANICS 2002-09-01       ARTS512   
4    Bill Mumy     ANALYTICAL MECHANICS 2002-09-01       ARTS512   

              course_name_ course_type  
0  BEHAVIORAL PHARMACOLOGY           F  
1  COMPUTER LINEAR ALGEBRA           F  
2     ANALYTICAL MECHANICS           F  
3     ANALYTICAL MECHANICS           F  
4     ANALYTICAL MECHANICS           F  


In [287]:
# Display the shape of the merged dataframe
print(f"Shape of merged dataframe: {merged_df.shape}")

Shape of merged dataframe: (2386, 6)


## Create a data frame with student names as the index, course numbers as columns, and if the student registered a course as values (0, 1)

In [290]:
# Create a pivot table
pivot_df = pd.pivot_table(merged_df, values='semester', index='student_name', 
                          columns='course_number', aggfunc='count', fill_value=0)

In [292]:
# Convert to binary (0 or 1)
pivot_df = (pivot_df > 0).astype(int)

In [294]:
# Display part of the final data frame
print("Part of the Final Data Frame:")
print(pivot_df.iloc[:4, :2])

Part of the Final Data Frame:
course_number  ARTS400  ARTS401
student_name                   
ABella Abzug         0        0
Al Gore              0        0
Al Hirt              0        0
Al Roker             1        0
